# Assignment 3
This is an assignment for COMPSCI 690V Visual analytics. The dataset used in this assignment is from UCI machine learning repository (http://archive.ics.uci.edu/ml/datasets/Wholesale+customers). In this assignment, I will go through various methods of clustering the data.

In [131]:
import pandas
from bokeh.layouts import widgetbox,row, Column
from bokeh.models.widgets import Select
from bokeh.models import ColumnDataSource
from bokeh.plotting import output_notebook, show, Figure
from bokeh.io import push_notebook
from bokeh.palettes import Accent
from ipywidgets import interactive, Box, Layout,HBox, VBox
from IPython.display import display
import ipywidgets as widgets
from sklearn import cluster
from sklearn.preprocessing import StandardScaler
import numpy
import warnings
warnings.filterwarnings('ignore')



output_notebook()
data = pandas.read_csv('Wholesale customers data.csv')
data.head()

Loading BokehJS ...

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,2,3,12669,9656,7561,214,2674,1338
1,2,3,7057,9810,9568,1762,3293,1776
2,2,3,6353,8808,7684,2405,3516,7844
3,1,3,13265,1196,4221,6404,507,1788
4,2,3,22615,5410,7198,3915,1777,5185


In [133]:
xAxis = list(data.columns)
xAxis.remove('Channel')
xAxis.remove('Region')
yAxis = xAxis
channels = {'Horeca':1, 'Retail':2, 'All':0}
regions = {'Lisbon':1, 'Oporto':2, 'Other Regions': 3, 'All':0}
channelOptions = list(channels.keys())
regionOptions = list(regions.keys())


dbscan   = cluster.DBSCAN(eps=.2)
birch    = cluster.Birch(n_clusters=2)
means    = cluster.MiniBatchKMeans(n_clusters=2)
spectral = cluster.SpectralClustering(n_clusters=2, eigen_solver='arpack', affinity="nearest_neighbors")
algorithms = {'DBSCAN':dbscan, 'Birch': birch, 'K-Means':means, 'Specteral':spectral}
algorithmOptions = list(algorithms.keys())

algorithm = means
clusters = 2
selectedData = data;
dataset = selectedData.as_matrix(columns=[xAxis[0],yAxis[1]])
dataset = StandardScaler().fit_transform(dataset)
algorithm.fit(dataset)
y_pred = algorithm.predict(dataset)

    

plot1 = Figure(x_axis_label=xAxis[0], y_axis_label=yAxis[1], plot_width=450, plot_height=450,title='Analysis of different clustering methods')
plot2 = Figure(x_axis_label=xAxis[0], y_axis_label=yAxis[1], plot_width=450, plot_height=450,title='Analysis of different clustering methods')

clusterPlots1 = []
clusterPlots2 = []
indicies = numpy.where(y_pred == -1)
source =selectedData.iloc[indicies]
clusterPlots1.append(plot1.circle(x=source[xAxis[0]],y=source[yAxis[1]],size=10,alpha=.5,color='grey')) 
clusterPlots2.append(plot2.circle(x=source[xAxis[0]],y=source[yAxis[1]],size=10,alpha=.5,color='grey')) 
for i in range(0,clusters):
    indicies = numpy.where(y_pred == i)
    source =selectedData.iloc[indicies]
    clusterPlots1.append(plot1.circle(x=source[xAxis[0]], y= source[yAxis[1]],size=10,alpha=.5,color=Accent[3][i]))
    clusterPlots2.append(plot2.circle(x=source[xAxis[0]], y= source[yAxis[1]],size=10,alpha=.5,color=Accent[3][i]))


graph = show(row(plot1,plot2), notebook_handle=True)

def drawGraph(channel, region, xAxis,yAxis,algorithm1,algorithm2):
    selectedData = data
    if(channels[channel] != 0):
        selectedData = data.loc[(data['Channel'] == channels[channel])]
    if(regions[region] != 0):
        selectedData =  selectedData.loc[data['Region'] == regions[region]]
    algorithm1 = algorithms[algorithm1]
    algorithm2 = algorithms[algorithm2]
    dataset = selectedData.as_matrix(columns=[xAxis,yAxis])
    dataset = StandardScaler().fit_transform(dataset)
    algorithm1.fit(dataset)
    algorithm2.fit(dataset)
    y_pred1 = algorithm1.labels_.astype(numpy.int)
    y_pred2 = algorithm2.labels_.astype(numpy.int)
    for i in range(0,clusters):
        indicies = numpy.where(y_pred1 == i)
        source =selectedData.iloc[indicies]
        clusterPlots1[i+1].data_source.data = {'x':source[xAxis],'y':source[yAxis]}
    indicies = numpy.where(y_pred1 == -1)
    source =selectedData.iloc[indicies]
    clusterPlots1[0].data_source.data = {'x':source[xAxis],'y':source[yAxis]}
    
    for i in range(0,clusters):
        indicies = numpy.where(y_pred2 == i)
        source =selectedData.iloc[indicies]
        clusterPlots2[i+1].data_source.data = {'x':source[xAxis],'y':source[yAxis]}
    indicies = numpy.where(y_pred2 == -1)
    source =selectedData.iloc[indicies]
    clusterPlots2[0].data_source.data = {'x':source[xAxis],'y':source[yAxis]}

    plot1.xaxis.axis_label = xAxis
    plot1.yaxis.axis_label = yAxis
    plot2.xaxis.axis_label = xAxis
    plot2.yaxis.axis_label = yAxis
    push_notebook(graph)

    
channelDropdown = widgets.Dropdown(options=channelOptions,value=channelOptions[2],description='Channel:') 
regionDropdown = widgets.Dropdown(options=regionOptions,value=regionOptions[3],description='Region:')
xAxisDropdown = widgets.Dropdown(options=xAxis,value=xAxis[0],description='X-Axis:')
yAxisDropdown = widgets.Dropdown(options=yAxis,value=yAxis[1],description='Y-Axis:')
algorithmDropdown1 = widgets.Dropdown(options=algorithmOptions, value=algorithmOptions[2],description='Clustering Algorithm:')
algorithmDropdown2 = widgets.Dropdown(options=algorithmOptions, value=algorithmOptions[2],description='Clustering Algorithm:')

output = interactive(drawGraph, channel=channelDropdown, region=regionDropdown, xAxis=xAxisDropdown,yAxis=yAxisDropdown, 
                     algorithm1 = algorithmDropdown1, algorithm2 = algorithmDropdown2)

box_layout = Layout(display='flex',
                    flex_flow='row',
                    justify_content='space-between',
                    width='85%')
VBox([HBox([output.children[0],output.children[1],output.children[2],output.children[3]]),
      Box([output.children[4],output.children[5]],layout=box_layout)])





Note: In the above graphs, blue and green represent different clusters. For the sake of simplicity, we have taken the maximum clusters as 2. The grey dots are not included in any clusters. As we can see, Birch clustering gives the best result